In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar10-python/cifar-10-python.tar.gz
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_1
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_2
/kaggle/input/cifar10-python/cifar-10-batches-py/batches.meta
/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_3
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_5
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_4
/kaggle/input/cifar10-python/cifar-10-batches-py/readme.html


In [5]:
import torch.nn as nn
import torch 
import torch.nn.functional as F 

In [6]:
from torch.utils.data import DataLoader, TensorDataset

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


train_batch = {}
test_batch = unpickle('/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch')
for i in range(5):
    train_batch[f'batch_{i+1}'] = unpickle("/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_%d"%(i + 1))


def get_dataset(batch_dict,test_dict):


    training_image = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'data']),torch.tensor(batch_dict['batch_2'][b'data']),torch.tensor(batch_dict['batch_3'][b'data']),torch.tensor(batch_dict['batch_4'][b'data']),torch.tensor(batch_dict['batch_5'][b'data'])),0),dtype = torch.float32)
    training_labels = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'labels']),torch.tensor(batch_dict['batch_2'][b'labels']),torch.tensor(batch_dict['batch_3'][b'labels']),torch.tensor(batch_dict['batch_4'][b'labels']),torch.tensor(batch_dict['batch_5'][b'labels']))))     

    test_images = torch.tensor(test_dict[b'data'],dtype = torch.float32)
    test_labels = torch.tensor(test_dict[b'labels'])

    training_image = training_image.view(50000,3,32,32)
    test_images = test_images.view(test_images.shape[0],3,32,32)

    return training_image,training_labels,test_images,test_labels


def make_batches(batch_size=16):
    training_image,training_labels,test_images,test_labels = get_dataset(train_batch,test_batch)
    train_dataset = TensorDataset(training_image,training_labels)
    test_dataset = TensorDataset(test_images,test_labels)
    train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
    test_loader = DataLoader(test_dataset,shuffle =True,batch_size=batch_size)

    return train_loader,test_loader

In [7]:
import torch 
import torch.nn as nn
def training(model,criterion,optimizer,train_loader,epochs=10,device='cuda'):
    training_loss = 0.0
    training_accuracy = 0
    for epoch in range(epochs):
        for i,(images,labels) in enumerate(train_loader):
            images= images.to(device)
            labels = labels.to(device)
            output = model(images)

            loss = criterion(output,labels)

            # training_loss += loss.item()
            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            _, predicted = torch.max(output.data, 1)

            training_accuracy += (predicted == labels).sum().item()
            if ((i+1)%100 == 0):
                print(f'epoch:{epoch + 1},loss:{loss.item():.4f}')
        print(f'accuracy:{training_accuracy*100/50000}')
        training_accuracy = 0    

#     torch.save(model.state_dict(),'/kaggle/working/model.pth')

    return model


def evaluation(model,test_loader,device='cuda'):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        test_loss = 0.0
        test_accuracy = 0
        for i,(images, labels) in enumerate(test_loader):
            images= images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
            
        acc = 100.0 * n_correct / n_samples
        
        print(f'Accuracy of the network on the 10000 test images: {acc} %')

In [8]:
class Resnet(nn.Module):

  def __init__ (self):

    super(Resnet,self).__init__()

    self.pad = nn.ZeroPad2d(2)

    self.Conv0 = nn.Conv2d(3,64,3,padding=1)
    self.Conv1 = nn.Conv2d(64,64,3,padding=1)
    self.Conv2 = nn.Conv2d(64,128,3,padding=1)
    self.Conv3 = nn.Conv2d(128,128,3,padding=1)
    self.Conv4 = nn.Conv2d(128,256,3,padding=1)
    self.Conv5 = nn.Conv2d(256,256,3,padding=1)
    self.Conv6 = nn.Conv2d(256,512,3,padding=1)
    self.Conv7 = nn.Conv2d(512,512,3,padding=1)
    self.maxpool = nn.MaxPool2d(2)

    self.bnorm1 = nn.BatchNorm2d(64)
    self.bnorm2 = nn.BatchNorm2d(128)
    self.bnorm3 = nn.BatchNorm2d(256)
    self.bnorm4 = nn.BatchNorm2d(512)

    self.avgpool = nn.AvgPool2d(2)

    self.Linear1 = nn.Linear(512*2*2,512)
    self.Linear2 = nn.Linear(512,10)

    self.softmax = nn.Softmax(1)


  def forward(self,x) :

     x = self.Conv0(x)

     shortcut = x

     x = F.relu(self.Conv1(x))
     x = F.relu(self.bnorm1(self.Conv1(x)))
     x = x + shortcut

     shortcut = x

     x = F.relu(self.Conv1(x))
     x = F.relu(self.bnorm1(self.Conv1(x)))
     x = x + shortcut

     shortcut = F.relu(self.bnorm2(self.Conv2(x)))
     shortcut = self.maxpool(shortcut)

     x = F.relu(self.bnorm2(self.Conv2(x)))
     x = F.relu(self.bnorm2(self.Conv3(x)))
     x = self.maxpool(x)
     x = x + shortcut

     shortcut = x

     x = F.relu(self.bnorm2(self.Conv3(x)))
     x = F.relu(self.bnorm2(self.Conv3(x)))
     x = x + shortcut

     shortcut = F.relu(self.bnorm3(self.Conv4(x)))
     shortcut = self.maxpool(shortcut)

     x = F.relu(self.bnorm3(self.Conv4(x)))
     x = F.relu(self.bnorm3(self.Conv5(x)))
     x = self.maxpool(x)
     x = x + shortcut

     shortcut = x

     x = F.relu(self.bnorm3(self.Conv5(x)))
     x = F.relu(self.bnorm3(self.Conv5(x)))
     x = x + shortcut

     shortcut = F.relu(self.bnorm4(self.Conv6(x)))
     shortcut = self.maxpool(shortcut)

     x = F.relu(self.bnorm4(self.Conv6(x)))
     x = F.relu(self.bnorm4(self.Conv7(x)))
     x = self.maxpool(x)
     x = x + shortcut

     shortcut = x

     x = F.relu(self.bnorm4(self.Conv7(x)))
     x = F.relu(self.bnorm4(self.Conv7(x)))
     x = x + shortcut


     x = self.avgpool(x)

     x = x.view(x.shape[0],512*2*2)
     x = F.relu(self.Linear1(x))

     x = self.Linear2(x)
    #  x = self.softmax(x)


     return x








In [9]:
resnet = Resnet()

In [10]:
device = 'cuda'
resnet.to(device)

Resnet(
  (pad): ZeroPad2d((2, 2, 2, 2))
  (Conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Conv6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Conv7): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine

In [11]:
train_loader,test_loader = make_batches(128)

/tmp/ipykernel_25/53764489.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_image = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'data']),torch.tensor(batch_dict['batch_2'][b'data']),torch.tensor(batch_dict['batch_3'][b'data']),torch.tensor(batch_dict['batch_4'][b'data']),torch.tensor(batch_dict['batch_5'][b'data'])),0),dtype = torch.float32)
/tmp/ipykernel_25/53764489.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_labels = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'labels']),torch.tensor(batch_dict['batch_2'][b'labels']),torch.tensor(batch_dict['batch_3'][b'labels']),torch.tensor(batch_dict['batch_4'][b'labels']),torch.tensor(

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(resnet.parameters(),lr = 0.001)

In [13]:
new_model =  training(resnet,criterion,optimizer,train_loader,epochs = 15)

epoch:1,loss:1.7580
epoch:1,loss:1.7694
epoch:1,loss:1.4976
accuracy:35.298
epoch:2,loss:1.1806
epoch:2,loss:1.0136
epoch:2,loss:1.0096
accuracy:60.976
epoch:3,loss:0.7609
epoch:3,loss:1.0184
epoch:3,loss:0.7242
accuracy:72.35
epoch:4,loss:0.6300
epoch:4,loss:0.5452
epoch:4,loss:0.6822
accuracy:78.494
epoch:5,loss:0.6107
epoch:5,loss:0.7032
epoch:5,loss:0.5154
accuracy:82.508
epoch:6,loss:0.4169
epoch:6,loss:0.4798
epoch:6,loss:0.4068
accuracy:85.446
epoch:7,loss:0.3701
epoch:7,loss:0.3757
epoch:7,loss:0.2792
accuracy:88.256
epoch:8,loss:0.3544
epoch:8,loss:0.2853
epoch:8,loss:0.2996
accuracy:89.824
epoch:9,loss:0.1817
epoch:9,loss:0.2402
epoch:9,loss:0.2097
accuracy:91.902
epoch:10,loss:0.2561
epoch:10,loss:0.1679
epoch:10,loss:0.1333
accuracy:93.274
epoch:11,loss:0.2046
epoch:11,loss:0.2217
epoch:11,loss:0.1701
accuracy:94.552
epoch:12,loss:0.1000
epoch:12,loss:0.1390
epoch:12,loss:0.1255
accuracy:95.798
epoch:13,loss:0.0602
epoch:13,loss:0.1062
epoch:13,loss:0.0681
accuracy:96.298
e

In [14]:
evaluation(resnet,test_loader)

Accuracy of the network on the 10000 test images: 87.07 %
